[Source](https://github.com/chewzzz1014/TensorFlow-Pokemon-Course)

In [1]:
!pip install tensorflow

In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [19]:
import os
os.listdir("./")

['.ipynb_checkpoints',
 '.jovianrc',
 'pokemon_alopez247.csv',
 'tensorflow-intro.ipynb']

In [20]:
path = "./pokemon_alopez247.csv"
df = pd.read_csv(path)
df.head()

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,...,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,...,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,...,Green,True,0.875,Monster,Grass,False,0.99,13.0,45,quadruped
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,...,Green,True,0.875,Monster,Grass,True,2.01,100.0,45,quadruped
3,4,Charmander,Fire,NaN,309,39,52,43,60,50,...,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
4,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,...,Red,True,0.875,Monster,Dragon,False,1.09,19.0,45,bipedal_tailed


In [21]:
df.columns

Index(['Number', 'Name', 'Type_1', 'Type_2', 'Total', 'HP', 'Attack',
       'Defense', 'Sp_Atk', 'Sp_Def', 'Speed', 'Generation', 'isLegendary',
       'Color', 'hasGender', 'Pr_Male', 'Egg_Group_1', 'Egg_Group_2',
       'hasMegaEvolution', 'Height_m', 'Weight_kg', 'Catch_Rate',
       'Body_Style'],
      dtype='object')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 721 entries, 0 to 720
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Number            721 non-null    int64  
 1   Name              721 non-null    object 
 2   Type_1            721 non-null    object 
 3   Type_2            350 non-null    object 
 4   Total             721 non-null    int64  
 5   HP                721 non-null    int64  
 6   Attack            721 non-null    int64  
 7   Defense           721 non-null    int64  
 8   Sp_Atk            721 non-null    int64  
 9   Sp_Def            721 non-null    int64  
 10  Speed             721 non-null    int64  
 11  Generation        721 non-null    int64  
 12  isLegendary       721 non-null    bool   
 13  Color             721 non-null    object 
 14  hasGender         721 non-null    bool   
 15  Pr_Male           644 non-null    float64
 16  Egg_Group_1       721 non-null    object 
 1

In [24]:
df = df[['isLegendary','Generation', 'Type_1', 'Type_2', 'HP', 'Attack', 'Defense', 'Sp_Atk', 'Sp_Def', 'Speed','Color','Egg_Group_1','Height_m','Weight_kg','Body_Style']]

## Categorical => Numerical

In [25]:
# isLegendary only has value of True/False
df["isLegendary"] = df["isLegendary"].astype(int)

In [26]:
# one-hot-encoding for other categorical features
def dummy_creation(df, cat):
    for i in cat:
        df_dummy = pd.get_dummies(df[i])
        df = pd.concat([df, df_dummy], axis=1)
        df = df.drop(i, axis=1)
    return (df)

cat_features = df.select_dtypes(include="object").columns
df = dummy_creation(df, list(cat_features))

In [27]:
df.head()

,isLegendary,Generation,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Height_m,Weight_kg,...,head_base,head_legs,head_only,insectoid,multiple_bodies,quadruped,serpentine_body,several_limbs,two_wings,with_fins
0,0,1,45,49,49,65,65,45,0.71,6.9,...,0,0,0,0,0,1,0,0,0,0
1,0,1,60,62,63,80,80,60,0.99,13.0,...,0,0,0,0,0,1,0,0,0,0
2,0,1,80,82,83,100,100,80,2.01,100.0,...,0,0,0,0,0,1,0,0,0,0
3,0,1,39,52,43,60,50,65,0.61,8.5,...,0,0,0,0,0,0,0,0,0,0
4,0,1,58,64,58,80,65,80,1.09,19.0,...,0,0,0,0,0,0,0,0,0,0


## Split Data into Train Set and Test Set

In [28]:
# split into train set and test set
def train_test_splitter(df, label_col):
    df_train = df.loc[df[label_col] != 1]
    df_test = df.loc[df[label_col] == 1]
    
    df_train = df_train.drop(label_col, axis=1)
    df_test = df_test.drop(label_col, axis=1)
    
    return (df_train, df_test)

df_train, df_test = train_test_splitter(df, "Generation")


# split into  X (data) and y (label) from df_train, df_test
def label_delimiter(df_train, df_test, label):
    train_data = df_train.drop(label, axis=1).values
    train_labels = df_train[label].values
    
    test_data = df_test.drop(label, axis=1).values
    test_labels = df_test[label].values
    
    # equalvalent to X_train, y_train, X_test, y_test
    return(train_data, train_labels, test_data, test_labels)

train_data, train_labels, test_data, test_labels = label_delimiter(df_train, df_test, "isLegendary")

In [29]:
print(f"Train Data Shape {train_data.shape}")
print(f"Train Labels Shape {train_labels.shape}")
print(f"Test Data Shape {test_data.shape}")
print(f"Test Labels Shape {test_labels.shape}")

Train Data Shape (570, 83)
Train Labels Shape (570,)
Test Data Shape (151, 83)
Test Labels Shape (151,)


## Normalize Data

In [45]:
def normalizer(train_data, test_data):
    train_data = preprocessing.MinMaxScaler().fit_transform(train_data)
    test_data = preprocessing.MinMaxScaler().fit_transform(test_data)
    return (train_data, test_data)

train_data, test_data = normalizer(train_data, test_data)

## Creating Model
- Using Keras (an API for Tensorflow)
- The model is built upon layers
- The number associated w/ the layers is the number of nuerons in it
- First layer: 
  - ReLU (Rectified Linear Unit)
  - input_size: Shape of an entry in the dataset
- Second Layer:
  - Type of logistic regression done for situations with multiple cases

In [49]:
length = train_data.shape[1]

model = keras.Sequential()
model.add(keras.layers.Dense(500, activation="relu", input_shape=[length, ]))
model.add(keras.layers.Dense(2, activation="softmax"))

## Compile Data
- Feeding parameters to model.compile
- Optimizer: Determines how model us updated as it gains information
- Loss Function: Measures how accurate the model is as it trains 
- Metrics: Specifies which information it provides can analyze the model
- SGD: Stochastic Gradient Descent(SGD) Optimization Algorithm

In [50]:
model.compile(optimizer='sgd', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Fit Data to Model

In [51]:
model.fit(train_data, train_labels, epochs=400)

Epoch 1/400
18/18 [==============================] - 1s 5ms/step - loss: 0.6375 - accuracy: 0.7123
Epoch 2/400
18/18 [==============================] - 0s 4ms/step - loss: 0.4753 - accuracy: 0.9263
Epoch 3/400
18/18 [==============================] - 0s 4ms/step - loss: 0.3941 - accuracy: 0.9263
Epoch 4/400
18/18 [==============================] - 0s 4ms/step - loss: 0.3497 - accuracy: 0.9263
Epoch 5/400
18/18 [==============================] - 0s 4ms/step - loss: 0.3231 - accuracy: 0.9263
Epoch 6/400
18/18 [==============================] - 0s 4ms/step - loss: 0.3057 - accuracy: 0.9263
Epoch 7/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2933 - accuracy: 0.9263
Epoch 8/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2840 - accuracy: 0.9263
Epoch 9/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2766 - accuracy: 0.9263
Epoch 10/400
18/18 [==============================] - 0s 4ms/step - loss: 0.2705 - accuracy: 0.9263
Epoch 11/

18/18 [==============================] - 0s 3ms/step - loss: 0.0811 - accuracy: 0.9842
Epoch 84/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0801 - accuracy: 0.9860
Epoch 85/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0791 - accuracy: 0.9877
Epoch 86/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0781 - accuracy: 0.9877
Epoch 87/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0772 - accuracy: 0.9877
Epoch 88/400
18/18 [==============================] - 0s 7ms/step - loss: 0.0764 - accuracy: 0.9877
Epoch 89/400
18/18 [==============================] - 0s 5ms/step - loss: 0.0755 - accuracy: 0.9912
Epoch 90/400
18/18 [==============================] - 0s 6ms/step - loss: 0.0746 - accuracy: 0.9912
Epoch 91/400
18/18 [==============================] - 0s 7ms/step - loss: 0.0738 - accuracy: 0.9912
Epoch 92/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0730 - accuracy: 0.9912
Epoch 93/400


18/18 [==============================] - 0s 3ms/step - loss: 0.0432 - accuracy: 0.9947
Epoch 165/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0429 - accuracy: 0.9947
Epoch 166/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0427 - accuracy: 0.9947
Epoch 167/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0425 - accuracy: 0.9947
Epoch 168/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0423 - accuracy: 0.9947
Epoch 169/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0421 - accuracy: 0.9947
Epoch 170/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0419 - accuracy: 0.9947
Epoch 171/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0417 - accuracy: 0.9947
Epoch 172/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0414 - accuracy: 0.9947
Epoch 173/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0412 - accuracy: 0.9947
Epoc

18/18 [==============================] - 0s 3ms/step - loss: 0.0308 - accuracy: 0.9965
Epoch 246/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0307 - accuracy: 0.9965
Epoch 247/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0306 - accuracy: 0.9965
Epoch 248/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0304 - accuracy: 0.9965
Epoch 249/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0304 - accuracy: 0.9965
Epoch 250/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0302 - accuracy: 0.9965
Epoch 251/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0301 - accuracy: 0.9965
Epoch 252/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0300 - accuracy: 0.9965
Epoch 253/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0299 - accuracy: 0.9965
Epoch 254/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0298 - accuracy: 0.9965
Epoc

18/18 [==============================] - 0s 3ms/step - loss: 0.0238 - accuracy: 0.9965
Epoch 327/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0237 - accuracy: 0.9965
Epoch 328/400
18/18 [==============================] - 0s 3ms/step - loss: 0.0237 - accuracy: 0.9965
Epoch 329/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0236 - accuracy: 0.9965
Epoch 330/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0235 - accuracy: 0.9965
Epoch 331/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0235 - accuracy: 0.9965
Epoch 332/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0234 - accuracy: 0.9965
Epoch 333/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0233 - accuracy: 0.9965
Epoch 334/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0233 - accuracy: 0.9965
Epoch 335/400
18/18 [==============================] - 0s 4ms/step - loss: 0.0232 - accuracy: 0.9965
Epoc

## Evaluate Model
- model.evaluate: Evaluate how strong the model is with the test data

In [52]:
loss_value, accuracy_value = model.evaluate(test_data, test_labels)
print(f"Our test accuracy was {accuracy_value}")

5/5 [==============================] - 0s 4ms/step - loss: 0.0354 - accuracy: 0.9801
Our test accuracy was 0.9801324605941772


## Predict

In [54]:
def predictor(test_data, test_labels, index):
    prediction = model.predict(test_data)
    
    if np.argmax(prediction[index]) == test_labels[index]:
        print(f'This was correctly predicted to be a \"{test_labels[index]}\"!')
    else:
        print(f'This was incorrectly predicted to be a \"{np.argmax(prediction[index])}\". It was actually a \"{test_labels[index]}\".')
        return(prediction)
    
predictor(test_data, test_labels, 149)

5/5 [==============================] - 0s 3ms/step
This was correctly predicted to be a "1"!
